In [9]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append("../../")
figures_output_path = "c:\\temp\\"
import pylab

import simulationengine.simulationRunner as simulationRunner

#continuous modules
import continuousSingleCommodity.link
import continuousSingleCommodity.originNode
import continuousSingleCommodity.destinationNode
import continuousSingleCommodity.mergeNode
import continuousSingleCommodity.oneToOneNode
import continuousSingleCommodity.divergeNode

import discrete.link
import discrete.originNode
import discrete.destinationNode
import discrete.oneToOneNode
import discrete.divergeNode
import discrete.mergeNode
import discrete.signalizedNode
import discrete.generalNodeModel
import trafficsignal.signalPlan

import demand.trip

import discrete.matsimLink

# One to One Case

In [10]:
after_dem = 0.2
demands_one_one = [1.0, 1.0, 1.0, 1.0, 1.0]
demands_one_one.extend([after_dem for _ in range(10)])

total_time = 150
def run_continuous(time_step):
    inbound_link = continuousSingleCommodity.link.Link(length=150, vf=30.0, w=6.0, kj=0.2)
    outbound_link = continuousSingleCommodity.link.Link(length=150, vf=30.0, w=6.0, kj=0.1)

    #demands = [1.0, 0.4, 0.3]
    origin_node = continuousSingleCommodity.originNode.OriginNode(1, inbound_link, demands_one_one)
    oneOneNode = continuousSingleCommodity.oneToOneNode.OneToOneNode(2, inbound_link, outbound_link)
    destination_node = continuousSingleCommodity.destinationNode.DestinationNode(3, outbound_link)

    links = [inbound_link, outbound_link]
    nodes = [origin_node, oneOneNode, destination_node]

    sim = simulationRunner.SimulationRunner(links = links, nodes = nodes, time_step = time_step, total_time=total_time)
    sim.run()
    return sim

def run_discrete(time_step):
    inbound_link = discrete.link.Link(length=150, vf=30.0, w=6.0, kj=0.2, initial_capacity=0)
    outbound_link = discrete.link.Link(length=150, vf=30.0, w=6.0, kj=0.1, initial_capacity=0)

    #demands = [1.0, 0.4, 0.3]
    trips = demand.trip.Trip.from_continuous_demand(demands_one_one, total_time, [1,2])
    origin_node = discrete.originNode.OriginNode(1, inbound_link, trips)
    oneOneNode = discrete.oneToOneNode.OneToOneNode(2, inbound_link, outbound_link)
    destination_node = discrete.destinationNode.DestinationNode(3, outbound_link)

    links = [inbound_link, outbound_link]
    nodes = [origin_node, oneOneNode, destination_node]

    sim = simulationRunner.SimulationRunner(links = links, nodes = nodes, time_step = time_step, total_time=total_time)
    sim.run()
    return sim


In [ ]:
result_continuous = run_continuous(1.0)

results_by_tstep = {}
for tstep in [1.0, 3.0, 6.0]:
    results_by_tstep[tstep] = run_discrete(tstep)

In [ ]:
import matplotlib
matplotlib.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
    }
)


styles = [
        {'color': 'black',
     'linestyle': '-',
     'linewidth': 3,
     'marker_size': None,
        'marker': 'o',
     'sim': result_continuous,
        't_step': 1.0,
    'alpha': 0.4,
     'label': 'C-$\\Delta t=1$ s'},
     {'color': '#D81B60',
     'linestyle': 'solid',
     'linewidth': 1,
     't_step': 1.0,
     'sim': results_by_tstep[1.0],
     'marker_size': 1,
     'marker': None,
     'alpha': None,
     'label': 'D-$\\Delta t=1$ s'},
     {'color': '#1E88E5',
     'linestyle': 'solid',
     'linewidth': 2,
    'marker': None,
     'marker_size': 3,
    'sim': results_by_tstep[3.0],
    't_step': 3.0,
    'alpha': None,
     'label': 'D-$\\Delta t=3$ s'},
     {'color': '#FFC107',
     'linestyle': '-',
     'linewidth': 2,
    'marker_size': 3,
    'marker': None,
    'sim': results_by_tstep[6.0],
    'alpha': None,
    't_step': 6.0,
     'label': 'D-$\\Delta t=6$ s'},


]

f, axs = pylab.subplots(3, 2, figsize=(8,5.5), sharex=True)

for case in styles:
    sim = case['sim']
    tstep = case['t_step']
    if t < shift:
        s = 1+t
    else:
        s = shift

    if case['label'][0] != 'C':
        ts = sim.get_times(1)

        f1s = []
        f2s = []
        f3s = []
        raw_f1s = []
        raw_f2s = []
        raw_f3s = []


        f1s.append((sim.links[0].cumulative_inflows[t+1]-sim.links[0].cumulative_inflows[t-s+1])/s)
        f2s.append((sim.links[1].cumulative_inflows[t+1]-sim.links[1].cumulative_inflows[t-s+1])/s)
        f3s.append((sim.links[1].cumulative_outflows[t+1]-sim.links[1].cumulative_outflows[t-s+1])/s)

        raw_f1s = [sim.links[0].cumulative_inflows[t+1]-sim.links[0].cumulative_inflows[t] for t in range(len(sim.get_times()))]
        raw_f2s = [sim.links[1].cumulative_inflows[t+1]-sim.links[1].cumulative_inflows[t] for t in range(len(sim.get_times()))]
        raw_f3s = [sim.links[1].cumulative_outflows[t+1]-sim.links[1].cumulative_outflows[t] for t in range(len(sim.get_times()))]


    else:
        f1s = [sim.links[0].cumulative_inflows[t+1]-sim.links[0].cumulative_inflows[t] for t in range(len(sim.get_times()))]
        f2s = [sim.links[1].cumulative_inflows[t+1]-sim.links[1].cumulative_inflows[t] for t in range(len(sim.get_times()))]
        f3s = [sim.links[1].cumulative_outflows[t+1]-sim.links[1].cumulative_outflows[t] for t in range(len(sim.get_times()))]

    #errrors
    if case['label'][0] != 'C':
        axs[1,0].stairs([sim.links[0].cumulative_inflows[t]-result_continuous.links[0].cumulative_inflows[t*int(tstep)] for t in range(len(ts))],
                        sim.get_times(2),
                    color = case['color'], linestyle=case['linestyle'], linewidth=case['linewidth'])
        
        axs[1,1].stairs([sim.links[1].cumulative_inflows[t]-result_continuous.links[1].cumulative_inflows[t*int(tstep)] for t in range(len(ts))],
                        sim.get_times(2),
            color = case['color'], linestyle=case['linestyle'], linewidth=case['linewidth'])
    
        axs[2,0].step(sim.get_times(0), raw_f1s, label=case['label'], color = case['color'], linestyle= case['linestyle'], linewidth=case['linewidth'], marker=case['marker'], markersize=case['marker_size'])
        axs[2,1].step(sim.get_times(0), raw_f2s, label=case['label'], color = case['color'], linestyle=case['linestyle'], linewidth=case['linewidth'], marker=case['marker'], markersize=case['marker_size'])
    else:
        axs[2,0].step(sim.get_times(), [el/tstep for el in f1s], label=case['label'], color = case['color'], linestyle=case['linestyle'], linewidth=2, alpha=0.6, markersize=1)
        axs[2,1].step(sim.get_times(), [el/tstep for el in f2s], label=case['label'], color = case['color'], linestyle=case['linestyle'], linewidth=2, alpha=0.6, markersize=1)
    
    n_v = [sim.links[0].cumulative_inflows[t]-sim.links[0].cumulative_outflows[t] for t in range(len(sim.get_times(1)))]
    if case['label'][0] != 'C':
        axs[0,0].step(sim.get_times(1), 
            sim.links[0].cumulative_inflows, label=case['label'], color = case['color'], linestyle=case['linestyle'], 
            linewidth=case['linewidth'], alpha=case['alpha'])
        axs[0,0].step(sim.get_times(1), 
                sim.links[1].cumulative_inflows, color = case['color'], linestyle=case['linestyle'], 
                linewidth=case['linewidth'], alpha=case['alpha'])
        axs[0,1].step(sim.get_times(1), n_v, label=case['label'], color = case['color'], linestyle=case['linestyle'], linewidth=case['linewidth'], alpha=case['alpha'])
    else:
        axs[0,0].plot(sim.get_times(1), 
            sim.links[0].cumulative_inflows, label=case['label'], color = case['color'], linestyle=case['linestyle'], 
            linewidth=case['linewidth']+3, alpha=case['alpha'])
        axs[0,0].plot(sim.get_times(1), 
                sim.links[1].cumulative_inflows, color = case['color'], linestyle=case['linestyle'], 
                linewidth=case['linewidth']+7, alpha=case['alpha'])
        axs[0,1].plot(sim.get_times(1), n_v, label=case['label'], color = case['color'], linestyle=case['linestyle'], linewidth=case['linewidth'], alpha=case['alpha'])
    
    
    
    

for i in range(3):
    for j in range(2):
        axs[i,j].grid()
        if i != 1:
            axs[i,j].set_ylim((0, None))
        axs[i,j].set_xlim(0, 150)

y_labels = [['Cum. Flows Link 1 \n $F_1(t)/G_1(t)$ (veh)', 'Vehicles in link 1 \n (veh)'],
            ['Cum. Inflow Error \r\n $F_1^c(t)-F_1^d(t)$ (veh)', 'Cum. Outflow Error \n $G_1^c(t)-G_1^d(t)$ (veh)'],
            ['Inflow Link 1 \n $f_1(t)$ - (veh)', 'Outflow Link 1 \n $g_1(t)$ - (veh)']
]

axs[0,0].annotate('Inflow', xy=(46, 47), xytext=(46, 47), xycoords='data', fontsize=fsize+4, rotation=30)
axs[0,0].annotate('Outflow', xy=(62, 15), xytext=(62, 15), xycoords='data', fontsize=fsize+4, rotation=30)

fsize=10
for i in range(3):
    for j in range(2):
        axs[i,j].set_ylabel(y_labels[i][j], fontsize=fsize)

axs[0,0].set_ylim((0,80))
axs[0,1].set_ylim((0,20))
axs[0,0].legend(fontsize=9)
axs[2,0].set_xlabel('time (s)')
axs[2,1].set_xlabel('Time (s)')
axs[2,0].set_ylim((0,6.3))
axs[2,1].set_ylim((0,6.3))
axs[1,0].set_yticks(list(range(-7,2)))

labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']
for i in range(3):
    for j in range(2):
        axs[i,j].set_title(labels[2*i+j], fontsize=fsize)

pylab.tight_layout()
pylab.savefig(figures_output_path+ "oneToOne.pdf", dpi=600)
pylab.show()

# Diverge Case

In [16]:
demands = [0.8 for _ in range(5)]
demands.extend([0.4 for _ in range(7)])
total_time = 120
def run_continuous_diverge():
    inbound_link = continuousSingleCommodity.link.Link(length=150, vf=30.0, w=6.0, kj=0.2)
    outbound_link_1 = continuousSingleCommodity.link.Link(length=150, vf=30.0, w=6.0, kj=0.1)
    outbound_link_2 = continuousSingleCommodity.link.Link(length=150, vf=30.0, w=6.0, kj=0.1)

    #demands = [1.0, 0.4, 0.3]
    origin_node = continuousSingleCommodity.originNode.OriginNode(1, inbound_link, demands)
    diverge_node = continuousSingleCommodity.divergeNode.DivergeNode(2, inbound_link, [outbound_link_1, outbound_link_2], [0.75, 0.25])
    
    destination_node_1 = continuousSingleCommodity.destinationNode.DestinationNode(3, outbound_link_1)
    destination_node_2 = continuousSingleCommodity.destinationNode.DestinationNode(4, outbound_link_2)

    links = [inbound_link, outbound_link_1, outbound_link_2]
    nodes = [origin_node, diverge_node, destination_node_1, destination_node_2]

    sim = simulationRunner.SimulationRunner(links = links, nodes = nodes, time_step = 1.0, total_time=total_time)
    sim.run()
    return sim

def run_discrete_diverge(time_step, random_route = False):
    inbound_link = discrete.link.Link(link_id = 1, length=150, vf=30.0, w=6.0, kj=0.2, initial_capacity=0)
    outbound_link_1 = discrete.link.Link(link_id = 2, length=150, vf=30.0, w=6.0, kj=0.1, initial_capacity=0)
    outbound_link_2 = discrete.link.Link(link_id = 3, length=150, vf=30.0, w=6.0, kj=0.1, initial_capacity=0)

    #demands = [1.0, 0.4, 0.3]
    trips = demand.trip.Trip.from_continuous_demand(demands, total_time, route_integer_share = {(1,2): 3, (1,3): 1},
                                             random_route=random_route)
    origin_node = discrete.originNode.OriginNode(1, inbound_link, trips)
    diverge_node = discrete.divergeNode.DivergeNode(2, inbound_link, [outbound_link_1, outbound_link_2])
    
    destination_node_1 = discrete.destinationNode.DestinationNode(3, outbound_link_1)
    destination_node_2 = discrete.destinationNode.DestinationNode(4, outbound_link_2)

    links = [inbound_link, outbound_link_1, outbound_link_2]
    nodes = [origin_node, diverge_node, destination_node_1, destination_node_2]

    sim = simulationRunner.SimulationRunner(links = links, nodes = nodes, time_step = time_step, total_time=total_time)
    sim.run()
    return sim

In [ ]:
sim_continuous = run_continuous_diverge()
sim_discrete = run_discrete_diverge(1.0)
sim_discrete_3 = run_discrete_diverge(3.0)

cases = [
    {'label': 'C', 'sim': sim_continuous, 'color': 'black', 'linestyle': '-', 'alpha': 0.5, 'linewidth': 3, 'row': 0},
    {'label': 'D-$\\Delta=1$s', 'sim': sim_discrete, 'color': '#A95AA1', 'linestyle': '-', 'alpha': None, 'linewidth': 1, 'row': 0},
    {'label': 'C', 'sim': sim_continuous, 'color': 'black', 'linestyle': '-', 'alpha': 0.5, 'linewidth': 3, 'row': 1},
    {'label': 'D-$\\Delta=3$s', 'sim': sim_discrete_3, 'color': '#A95AA1', 'linestyle': '-', 'alpha': None, 'linewidth': 1, 'row': 1},
]

f, axs = pylab.subplots(2,3, figsize=(8,5.5), sharex=True)

color_1 = '#601A4A'
color_2 = '#63ACBE'
color_3 = '#FAAC77'

for case in cases:
    sim = case['sim']
    times_p = sim.get_times(1)

    f0 = [sim.links[0].cumulative_outflows[t+1]-sim.links[0].cumulative_outflows[t] for t in range(len(times_p)-1)]
    f1 = [sim.links[1].cumulative_inflows[t+1]-sim.links[1].cumulative_inflows[t] for t in range(len(times_p)-1)]
    f2 = [sim.links[2].cumulative_inflows[t+1]-sim.links[2].cumulative_inflows[t] for t in range(len(times_p)-1)]
    f3 = [sim.links[0].cumulative_inflows[t+1]-sim.links[0].cumulative_inflows[t] for t in range(len(times_p)-1)]

    if case['label'][0]=='C':
        axs[case['row'],0].plot(times_p, sim.links[0].cumulative_outflows, color=color_1, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth']+3, label='$G_u(t)$')
        axs[case['row'],0].plot(times_p, sim.links[1].cumulative_inflows, color=color_2, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth']+3, label='$F_1(t)$')
        axs[case['row'],0].plot(times_p, sim.links[2].cumulative_inflows, color=color_3, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'], label='$F_2(t)$')

        for u in (1,2):
            axs[case['row'],u].plot(sim.get_times(0), f0, color=color_1, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'], label='$g_u(t)$')
            axs[case['row'],u].plot(sim.get_times(0), f1, color=color_2, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'], label='$f_1(t)$')
            axs[case['row'],u].plot(sim.get_times(0), f3, color='black', linestyle='--', alpha=case['alpha'], linewidth=case['linewidth'], label='$d(t)$')
            axs[case['row'],u].plot(sim.get_times(0), f2, color=color_3, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'], label='$f_2(t)$')
        #axs[case['row'],2].plot(sim.get_times(0), f1, color=color_3, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'])
        

    else:
        axs[case['row'],0].step(times_p, sim.links[0].cumulative_outflows, color=color_1, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=2*case['linewidth'])
        axs[case['row'],0].step(times_p, sim.links[1].cumulative_inflows, color=color_2, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=2*case['linewidth'])
        axs[case['row'],0].step(times_p, sim.links[2].cumulative_inflows, color=color_3, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'])

        axs[case['row'],1].step(sim.get_times(0), f1, color=color_2, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'], label='$f_1^d(t)$')
        axs[case['row'],2].step(sim.get_times(0), f2, color=color_3, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'], label='$f_2^d(t)$')

axs[1,0].set_xlabel('Time (s)')
axs[1,1].set_xlabel('Time (s)')
fsize=10
for i in range(2):
    axs[i,0].set_ylabel('Cum. Flows \n $G_1(t)$, $F_1(t)$, $G_2(t)$ (veh)', fontsize=fsize)
    axs[i,1].set_ylabel('Link 1 inflow \n $f_1(t)$ (veh)', fontsize=fsize)
    axs[i,2].set_ylabel('Link 2 Outflow \n $f_2(t)$ (veh)',     fontsize=fsize)

    if i==0:
        upper = 3
    else:
        upper = 3

    axs[i,1].set_ylim((-0.00, upper))
    axs[i,2].set_ylim((-0.00, upper))

    axs[i,0].set_ylim((0, 70))
    
    for j in range(3):
        axs[i,j].grid(True)
        loc = None
        if j > 0:
            loc = 'upper right'
        axs[i,j].legend(fontsize=7, loc=loc)

        axs[i,j].set_xlim((0,120))
        axs[i,j].set_xticks([0,20,40,60,80,100,120])


axs[0,1].set_title('$\\Delta t=1$ s')
axs[1,1].set_title('$\\Delta t=3$ s')

axs[0,0].grid(True)
#axs[0,0].legend()

titles = ['(a) $\\Delta t=1$ s', '(b) $\\Delta t=1$ s', '(c) $\\Delta t=1$ s', '(d) $\\Delta t=3$ s', '(e) $\\Delta t=3$ s', '(f) $\\Delta t=3$ s']
for i in range(2):
    for j in range(3):
        axs[i,j].set_title(titles[3*i+j], fontsize=fsize)

pylab.tight_layout()
pylab.savefig(figures_output_path + "diverge.pdf", dpi=600)
pylab.show()

In [ ]:
simulations = []
for i in range(20):
    simulations.append(run_discrete_diverge(1.0, random_route=True))

f, axs = pylab.subplots(1,1, figsize=(9,4.5))

for sim in simulations:
    times_p = sim.get_times(1)
    axs.plot(times_p, sim.links[0].cumulative_outflows, color=color_1, alpha=0.2)
    axs.plot(times_p, sim.links[1].cumulative_inflows, color=color_2, alpha=0.2)
    axs.plot(times_p, sim.links[2].cumulative_inflows, color=color_3, alpha=0.2)

axs.plot(times_p, sim_continuous.links[0].cumulative_outflows, color=color_1, linestyle='-', label='$G_u(t)$')
axs.plot(times_p, sim_continuous.links[1].cumulative_inflows, color=color_2, linestyle='-', label='$F_1(t)$')
axs.plot(times_p, sim_continuous.links[2].cumulative_inflows, color=color_3, linestyle='-', label='$G_2(t)$')

axs.plot(sim_discrete.get_times(1), sim_discrete.links[0].cumulative_outflows, color=color_1, linestyle='--')
axs.plot(sim_discrete.get_times(1), sim_discrete.links[1].cumulative_inflows, color=color_2, linestyle='--')
axs.plot(sim_discrete.get_times(1), sim_discrete.links[2].cumulative_inflows, color=color_3, linestyle='--')

#for i in range(3):
axs.grid(True)
axs.set_xlim(0, 120)
axs.set_ylim(0, 70)
axs.set_xlabel("Time (s)")

axs.set_ylabel("Cumulative Flows (veh)")
axs.legend()
pylab.tight_layout()
pylab.savefig(figures_output_path+"diverge_random.pdf", dpi=600)
pylab.show()
    

# Merge Case

In [21]:
dem_1 = [0.3 for _ in range(12)]

dem_2 = [0.3 for _ in range(12)]
for p in range(4,12):
    dem_2[p] = 0.1

def run_continuous_merge(priorities):
    inbound_link_1 = continuousSingleCommodity.link.Link(link_id=1,length=150, vf=30.0, w=6.0, kj=0.1)
    inbound_link_2 = continuousSingleCommodity.link.Link(link_id=2, length=150, vf=30.0, w=6.0, kj=0.1)
    outbound_link = continuousSingleCommodity.link.Link(link_id=3, length=150, vf=30.0, w=6.0, kj=0.1)

    origin_node_1 = continuousSingleCommodity.originNode.OriginNode(1, inbound_link_1, dem_1)
    origin_node_2 = continuousSingleCommodity.originNode.OriginNode(2, inbound_link_2, dem_2)
    merge_node = continuousSingleCommodity.mergeNode.MergeNode(3, [inbound_link_1, inbound_link_2], outbound_link, priorities)
    
    destination_node = continuousSingleCommodity.destinationNode.DestinationNode(4, outbound_link)

    links = [inbound_link_1, inbound_link_2, outbound_link]
    nodes = [origin_node_1, origin_node_2, merge_node, destination_node]

    sim = simulationRunner.SimulationRunner(links = links, nodes = nodes, time_step = 1.0, total_time=120)
    sim.run()
    return sim

def run_discrete_merge(priorities, time_step):
    inbound_link_1 = discrete.link.Link(link_id=1,length=150, vf=30.0, w=6.0, kj=0.1, initial_capacity=0)
    inbound_link_2 = discrete.link.Link(link_id=2, length=150, vf=30.0, w=6.0, kj=0.1, initial_capacity=0)
    outbound_link = discrete.link.Link(link_id=3, length=150, vf=30.0, w=6.0, kj=0.1, initial_capacity=0)

    origin_node_1 = discrete.originNode.OriginNode(1, inbound_link_1, demand.trip.Trip.from_continuous_demand(dem_1, 120))
    origin_node_2 = discrete.originNode.OriginNode(2, inbound_link_2, demand.trip.Trip.from_continuous_demand(dem_2, 120))
    merge_node = discrete.mergeNode.MergeNode(3, outbound_link, [inbound_link_1, inbound_link_2], priorities)
    
    destination_node = discrete.destinationNode.DestinationNode(4, outbound_link)

    links = [inbound_link_1, inbound_link_2, outbound_link]
    nodes = [origin_node_1, origin_node_2, merge_node, destination_node]

    sim = simulationRunner.SimulationRunner(links = links, nodes = nodes, time_step = time_step, total_time=120)
    sim.run()
    return sim

In [ ]:
cont_05 = run_continuous_merge([0.5, 0.5])
cont_075 = run_continuous_merge([0.75, 0.25])

disc_05 = run_discrete_merge([0, 1], 1.0)
disc_75 = run_discrete_merge([0, 0, 0, 1], 1.0)

cases = [
    {'label': 'C', 'sim': cont_05, 'color': 'black', 'linestyle': '-', 'alpha': 0.7, 'linewidth': 3, 'row': 0},
    {'label': 'D-$\\Delta=1s$', 'sim': disc_05, 'color': '#A95AA1', 'linestyle': '-', 'alpha': None, 'linewidth': 1, 'row': 0},
    {'label': 'C', 'sim': cont_075, 'color': 'black', 'linestyle': '-', 'alpha': 0.7, 'linewidth': 3, 'row': 1},
    {'label': 'D-$\\Delta=1s$', 'sim': disc_75, 'color': '#A95AA1', 'linestyle': '-', 'alpha': None, 'linewidth': 1, 'row': 1},
]

color_1 = '#601A4A'
color_2 = '#63ACBE'
color_3 = '#FAAC77'
f, axs = pylab.subplots(2,3, figsize=(8,5.5), sharex=True)

for case in cases:
    sim = case['sim']
    times_p = sim.get_times(1)

    f0 = [sim.links[0].cumulative_outflows[t+1]-sim.links[0].cumulative_outflows[t] for t in range(len(times_p)-1)]
    f1 = [sim.links[1].cumulative_outflows[t+1]-sim.links[1].cumulative_outflows[t] for t in range(len(times_p)-1)]
    f2 = [sim.links[2].cumulative_inflows[t+1]-sim.links[2].cumulative_inflows[t] for t in range(len(times_p)-1)]

    if case['label'][0]=='C':
        axs[case['row'],0].plot(times_p, sim.links[0].cumulative_outflows, color=color_1, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth']+3, label='$G_1(t)$')
        axs[case['row'],0].plot(times_p, sim.links[1].cumulative_outflows, color=color_2, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth']+3, label='$G_2(t)$')
        axs[case['row'],0].plot(times_p, sim.links[2].cumulative_inflows, color=color_3, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'], label='$F_u(t)$')

        axs[case['row'],1].plot(sim.get_times(0), f0, color=color_1, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'], label='$g_1(t)$')
        axs[case['row'],2].plot(sim.get_times(0), f1, color=color_2, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'], label='$g_2(t)$')
        #axs[case['row'],2].plot(sim.get_times(0), f1, color=color_3, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'])
        

    else:
        axs[case['row'],0].step(times_p, sim.links[0].cumulative_outflows, color=color_1, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=2*case['linewidth'])
        axs[case['row'],0].step(times_p, sim.links[1].cumulative_outflows, color=color_2, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=2*case['linewidth'])
        axs[case['row'],0].step(times_p, sim.links[2].cumulative_inflows, color=color_3, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'])

        axs[case['row'],1].step(sim.get_times(0), f0, color=color_1, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'], label='$g_1^d(t)$')
        axs[case['row'],2].step(sim.get_times(0), f1, color=color_2, linestyle=case['linestyle'], alpha=case['alpha'], linewidth=case['linewidth'], label='$g_2^d(t)$')


axs[1,0].set_xlabel('Time (s)')
axs[1,1].set_xlabel('Time (s)')
fsize=10
for i in range(2):
    axs[i,0].set_ylabel('Cum. Flows \n $G_1(t)$, $G_2(t)$, $F_3(t)$ (veh)', fontsize=fsize)
    axs[i,1].set_ylabel('Link 1 Outflow \n $f_1(t)$ (veh)', fontsize=fsize)
    axs[i,2].set_ylabel('Link 2 Outflow \n $f_2(t)$ (veh)',     fontsize=fsize)

    axs[i,1].set_ylim((-0.00, 2.0))
    axs[i,2].set_ylim((-0.00, 2.0))

    axs[i,0].set_ylim((0, 60))
    
    for j in range(3):
        axs[i,j].grid(True)
        axs[i,j].legend()

        axs[i,j].set_xlim((0,120))
        axs[i,j].set_xticks([0,20,40,60,80,100,120])


titles = ['(a) $\\alpha=0.5$, $\\mathbf{x}=[0, 1]$', '(b) $\\alpha=0.5$, $\\mathbf{x}=[0, 1]$', '(c) $\\alpha=0.5$, $\\mathbf{x}=[0, 1]$', 
          '(d) $\\alpha=0.75$, $\\mathbf{x}=[0, 0,0,1]$', '(e) $\\alpha=0.75$, $\\mathbf{x}=[0, 0,0,1]$', '(f) $\\alpha=0.75$, $\\mathbf{x}=[0, 0,0,1]$']

for i in range(2):
    for j in range(3):
        axs[i,j].set_title(titles[3*i+j], fontsize=fsize)

#axs[0,1].set_title('$\\alpha=0.5$, $\\mathbf{x}=[0, 1]$')
#axs[1,1].set_title('$\\alpha=0.75$, $\\mathbf{x}=[0, 0,0,1]$')

axs[0,0].grid(True)
axs[0,0].legend()

pylab.tight_layout()
pylab.savefig(figures_output_path+ "merge.pdf", dpi=600)
pylab.show()


# Comparison with MatSim Link

Link and parameters with the same structures but there is no state variable for capacity and it is used $int(C\Delta t)$ instead.

Here a comparison with continuous case and two different time-steps.

In [25]:
time_matsim = 150
demands_matsim = [1.0, 0.2, 0.2]
def run_matsim_discrete(time_step):
    inbound_link = discrete.matsimLink.MatsimLink(length=150, vf=30.0, w=7.0, kj=0.2, initial_capacity=0)
    outbound_link = discrete.matsimLink.MatsimLink(length=150, vf=30.0, w=7.0, kj=0.1, initial_capacity=0)

    #demands = [1.0, 0.4, 0.3]
    trips = demand.trip.Trip.from_continuous_demand(demands_matsim, time_matsim, [1,2])
    origin_node = discrete.originNode.OriginNode(1, inbound_link, trips)
    oneOneNode = discrete.oneToOneNode.OneToOneNode(2, inbound_link, outbound_link)
    destination_node = discrete.destinationNode.DestinationNode(3, outbound_link)

    links = [inbound_link, outbound_link]
    nodes = [origin_node, oneOneNode, destination_node]

    sim = simulationRunner.SimulationRunner(links = links, nodes = nodes, time_step = time_step, total_time=time_matsim)
    sim.run()
    return sim

def run_one_one_comparison_continuous(time_step):
    inbound_link = continuousSingleCommodity.link.Link(length=150, vf=30.0, w=7.0, kj=0.2)
    outbound_link = continuousSingleCommodity.link.Link(length=150, vf=30.0, w=7.0, kj=0.1)

    #demands = [1.0, 0.4, 0.3]
    origin_node = continuousSingleCommodity.originNode.OriginNode(1, inbound_link, demands_matsim)
    oneOneNode = continuousSingleCommodity.oneToOneNode.OneToOneNode(2, inbound_link, outbound_link)
    destination_node = continuousSingleCommodity.destinationNode.DestinationNode(3, outbound_link)

    links = [inbound_link, outbound_link]
    nodes = [origin_node, oneOneNode, destination_node]

    sim = simulationRunner.SimulationRunner(links = links, nodes = nodes, time_step = time_step, total_time=time_matsim)
    sim.run()
    return sim

def run_one_one_comparison_discrete(time_step):
    inbound_link = discrete.link.Link(length=150, vf=30.0, w=7.0, kj=0.2, initial_capacity=0)
    outbound_link = discrete.link.Link(length=150, vf=30.0, w=7.0, kj=0.1, initial_capacity=0)

    #demands = [1.0, 0.4, 0.3]
    trips = demand.trip.Trip.from_continuous_demand(demands_matsim, time_matsim, [1,2])
    origin_node = discrete.originNode.OriginNode(1, inbound_link, trips)
    oneOneNode = discrete.oneToOneNode.OneToOneNode(2, inbound_link, outbound_link)
    destination_node = discrete.destinationNode.DestinationNode(3, outbound_link)

    links = [inbound_link, outbound_link]
    nodes = [origin_node, oneOneNode, destination_node]

    sim = simulationRunner.SimulationRunner(links = links, nodes = nodes, time_step = time_step, total_time=time_matsim)
    sim.run()
    return sim


In [ ]:
sim_continuous = run_one_one_comparison_continuous(1.0)
sim_matsim_2 = run_matsim_discrete(2.0)
sim_matsim_4 = run_matsim_discrete(4.0)
sim_discrete_2 = run_one_one_comparison_discrete(2.0)
sim_discrete_4 = run_one_one_comparison_discrete(4.0)

styles = [
        {'color': 'black',
     'linestyle': '-',
     'linewidth': 3,
     'marker_size': None,
        'marker': 'o',
     'sim': sim_continuous,
        't_step': 1.0,
    'alpha': 0.4,
     'label': 'C-$\\Delta t=1$ s'},
     {'color': '#D81B60',
     'linestyle': 'solid',
     'linewidth': 1,
     't_step': 2.0,
     'sim': sim_discrete_2,
     'marker_size': 1,
     'marker': None,
     'alpha': None,
     'label': 'D-$\\Delta t=2$ s'},
     {'color': '#1E88E5',
     'linestyle': 'solid',
     'linewidth': 2,
    'marker': None,
     'marker_size': 3,
    'sim': sim_discrete_4,
    't_step': 4.0,
    'alpha': None,
     'label': 'D-$\\Delta t=4$ s'},
     {'color': '#FFC107',
     'linestyle': '--',
     'linewidth': 2,
    'marker_size': 3,
    'marker': None,
    'sim': sim_matsim_2,
    'alpha': None,
    't_step': 2.0,
     'label': '$\\hat{q}^{u,d}=\\lfloor C\\Delta t \\rfloor $ $\\Delta t=2$ s'},
     {'color': '#21B933',
     'linestyle': '--',
     'linewidth': 2,
    'marker_size': 3,
    'marker': None,
    'sim': sim_matsim_4,
    'alpha': None,
    't_step': 4.0,
     'label': '$\\hat{q}^{u,d}=\\lfloor C\\Delta t \\rfloor $ $\\Delta t=4$ s'}


]
fsize=5

f, axs = pylab.subplots(3, 2, figsize=(8,5.5), sharex=True)

for case in styles:
    sim = case['sim']
    tstep = case['t_step']


    if case['label'][0] != 'C':
        ts = sim.get_times(1)

        raw_f1s = []
        raw_f2s = []
        raw_f3s = []



        raw_f1s = [sim.links[0].cumulative_inflows[t+1]-sim.links[0].cumulative_inflows[t] for t in range(len(sim.get_times()))]
        raw_f2s = [sim.links[1].cumulative_inflows[t+1]-sim.links[1].cumulative_inflows[t] for t in range(len(sim.get_times()))]
        raw_f3s = [sim.links[1].cumulative_outflows[t+1]-sim.links[1].cumulative_outflows[t] for t in range(len(sim.get_times()))]


    else:
        f1s = [sim.links[0].cumulative_inflows[t+1]-sim.links[0].cumulative_inflows[t] for t in range(len(sim.get_times()))]
        f2s = [sim.links[1].cumulative_inflows[t+1]-sim.links[1].cumulative_inflows[t] for t in range(len(sim.get_times()))]
        f3s = [sim.links[1].cumulative_outflows[t+1]-sim.links[1].cumulative_outflows[t] for t in range(len(sim.get_times()))]

    #errrors
    if case['label'][0] != 'C':
        axs[1,0].stairs([sim.links[0].cumulative_inflows[t]-sim_continuous.links[0].cumulative_inflows[t*int(tstep)] for t in range(len(ts))],
                        sim.get_times(2),
                    color = case['color'], linestyle=case['linestyle'], linewidth=case['linewidth'])
        
        axs[1,1].stairs([sim.links[1].cumulative_inflows[t]-sim_continuous.links[1].cumulative_inflows[t*int(tstep)] for t in range(len(ts))],
                        sim.get_times(2),
            color = case['color'], linestyle=case['linestyle'], linewidth=case['linewidth'])
    
        axs[2,0].step(sim.get_times(0), raw_f1s, label=case['label'], color = case['color'], linestyle= case['linestyle'], linewidth=case['linewidth'], marker=case['marker'], markersize=case['marker_size'])
        axs[2,1].step(sim.get_times(0), raw_f2s, label=case['label'], color = case['color'], linestyle=case['linestyle'], linewidth=case['linewidth'], marker=case['marker'], markersize=case['marker_size'])
    else:
        axs[2,0].step(sim.get_times(), [el/tstep for el in f1s], label=case['label'], color = case['color'], linestyle=case['linestyle'], linewidth=2, alpha=0.6, markersize=1)
        axs[2,1].step(sim.get_times(), [el/tstep for el in f2s], label=case['label'], color = case['color'], linestyle=case['linestyle'], linewidth=2, alpha=0.6, markersize=1)
    
    n_v = [sim.links[0].cumulative_inflows[t]-sim.links[0].cumulative_outflows[t] for t in range(len(sim.get_times(1)))]
    if case['label'][0] != 'C':
        axs[0,0].step(sim.get_times(1), 
            sim.links[0].cumulative_inflows, label=case['label'], color = case['color'], linestyle=case['linestyle'], 
            linewidth=1, alpha=case['alpha'])
        axs[0,0].step(sim.get_times(1), 
                sim.links[1].cumulative_inflows, color = case['color'], linestyle=case['linestyle'], 
                linewidth=1, alpha=case['alpha'])
        axs[0,1].step(sim.get_times(1), n_v, label=case['label'], color = case['color'], linestyle=case['linestyle'], linewidth=case['linewidth'], alpha=case['alpha'])
    else:
        axs[0,0].plot(sim.get_times(1), 
            sim.links[0].cumulative_inflows, label=case['label'], color = case['color'], linestyle=case['linestyle'], 
            linewidth=1, alpha=case['alpha'])
        axs[0,0].plot(sim.get_times(1), 
                sim.links[1].cumulative_inflows, color = case['color'], linestyle=case['linestyle'], 
                linewidth=1, alpha=case['alpha'])
        axs[0,1].plot(sim.get_times(1), n_v, label=case['label'], color = case['color'], linestyle=case['linestyle'], linewidth=case['linewidth'], alpha=case['alpha'])
    
    
    
    

for i in range(3):
    for j in range(2):
        axs[i,j].grid()
        if i != 1:
            axs[i,j].set_ylim((0, None))
        axs[i,j].set_xlim(0, time_matsim)

y_labels = [['Cum. Flows Link 1 \n $F_1(t)/G_1(t)$ (veh)', 'Vehicles in link 1 \n (veh)'],
            ['Cum. Inflow Error \r\n $F_1^c(t)-F_1^d(t)$ (veh)', 'Cum. Outflow Error \n $G_1^c(t)-G_1^d(t)$ (veh)'],
            ['Inflow Link 1 \n $f_1(t)$ - (veh)', 'Outflow Link 1 \n $g_1(t)$ - (veh)']
]

axs[0,0].annotate('Inflow', xy=(46, 47), xytext=(46, 47), xycoords='data', fontsize=fsize+4, rotation=30)
axs[0,0].annotate('Outflow', xy=(62, 15), xytext=(62, 15), xycoords='data', fontsize=fsize+4, rotation=30)

fsize=10
for i in range(3):
    for j in range(2):
        axs[i,j].set_ylabel(y_labels[i][j], fontsize=fsize)

axs[0,0].set_ylim((0,70))
axs[0,1].set_ylim((0,25))
axs[0,0].legend(fontsize=7)
axs[2,0].set_xlabel('time (s)')
axs[2,1].set_xlabel('Time (s)')
axs[2,0].set_ylim((0,6.3))
axs[2,1].set_ylim((0,6.3))

graph_labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']
for i in range(3):
    for j in range(2):
        axs[i,j].set_title(graph_labels[2*i+j], fontsize=9)

pylab.tight_layout()
pylab.savefig(figures_output_path + "one_one_capacity.pdf", dpi=600)
pylab.show()

# Signalized Intersections
This case has no clear counterpart with the continuous model.

In [28]:
def run_discrete_signalized(dem_1, dem_2, dem_3):
    time_step = 1.0
    total_time = 240

    inbound_link_1 = discrete.link.Link(link_id=1,length=300, vf=30.0, w=6.0, kj=0.2, initial_capacity=0)
    inbound_link_2 = discrete.link.Link(link_id=2, length=300, vf=30.0, w=6.0, kj=0.1, initial_capacity=0)
    inbound_link_3 = discrete.link.Link(link_id=3, length=300, vf=30.0, w=6.0, kj=0.2, initial_capacity=0)

    outbound_link_5 = discrete.link.Link(link_id=5, length=300, vf=30.0, w=6.0, kj=0.2, initial_capacity=0)
    outbound_link_6 = discrete.link.Link(link_id=6, length=300, vf=30.0, w=6.0, kj=0.1, initial_capacity=0)
    outbound_link_7 = discrete.link.Link(link_id=7, length=300, vf=30.0, w=6.0, kj=0.2, initial_capacity=0)
    
    inbound_links = [inbound_link_1, inbound_link_2, inbound_link_3]
    outbound_links = [outbound_link_5, outbound_link_6, outbound_link_7]
    all_links = inbound_links + outbound_links




    origin_node_1 = discrete.originNode.OriginNode(1, inbound_link_1, demand.trip.Trip.from_continuous_demand(dem_1, total_time, route=[1,5]))
    origin_node_2 = discrete.originNode.OriginNode(2, inbound_link_2, demand.trip.Trip.from_continuous_demand(dem_2, total_time, route=[2,6]))
    origin_node_3 = discrete.originNode.OriginNode(3, inbound_link_3, demand.trip.Trip.from_continuous_demand(dem_3, total_time, route=[3,7]))

    protected_intervals = {
        1: [[0, 40]],
        2: [[0, 8]],
        3: [[10, 40]],
        4: [[10, 40]]
    }
    
    permitted_intervals = {
        1: [],
        2: [[8, 40]],
        3: [],
        4: []
    }

    stop_permit_intervals = {
        1: [],
        2: [],
        3: [],
        4: [[0,10], [40, 60]]
    }
    conflicting_map = {1: [], 2: [3,], 3: [2]}


    plan = trafficsignal.signalPlan.SignalPlan(4, 60, 0, protected_intervals, permitted_intervals, stop_permit_intervals)
    
    signal_node = discrete.signalizedNode.SignalizedNode(4, inbound_links, outbound_links, conflicting_map, plan)

    all_nodes = [origin_node_1, origin_node_2, origin_node_3, signal_node]

    for link in outbound_links:
        destination_node = discrete.destinationNode.DestinationNode(link.link_id+1, link)
        all_nodes.append(destination_node)

    

    sim = simulationRunner.SimulationRunner(links = all_links, nodes = all_nodes, time_step = time_step, total_time=total_time)
    sim.run()
    return sim

In [ ]:
dem_1 = [0.2, 0.2, 0.2, 0.2]
dem_2 = [0.15, 0.15, 0.15, 0.15]
dem_3 = [0.4, 0.4, 0.15, 0.15]
times_demands = [0, 60, 120, 180, 240]
sim = run_discrete_signalized(dem_1, dem_2, dem_3)

dem_2.insert(0, dem_2[0])
#dem_3.append(dem_3[-1])
dem_3.insert(0, dem_3[0])

times_p = sim.get_times(1)

#axs[0].plot(times_p, sim.links[0].cumulative_outflows, label='D1-out', color='blue')
#axs[1].plot(times_p, sim.links[1].cumulative_outflows, label='D2-out', color='blue')
#axs[2].plot(times_p, sim.links[2].cumulative_outflows, label='D3-out', color='blue')


color_1 = '#601A4A'
color_2 = '#63ACBE'
color_3 = '#FAAC77'
f, axs = pylab.subplots(2,1, figsize=(8,5.5), sharex=True)


f0 = [sim.links[0].cumulative_outflows[t+1]-sim.links[0].cumulative_outflows[t] for t in range(len(times_p)-1)]
f1 = [sim.links[1].cumulative_outflows[t+1]-sim.links[1].cumulative_outflows[t] for t in range(len(times_p)-1)]
f2 = [sim.links[2].cumulative_outflows[t+1]-sim.links[2].cumulative_outflows[t] for t in range(len(times_p)-1)]

q_0_fol = [sim.links[0].cumulative_inflows[t-10]-sim.links[0].cumulative_outflows[t] for t in range(10,len(times_p))]
q_0 = [0 for _ in range(9)]+q_0_fol

q_1_fol = [sim.links[1].cumulative_inflows[t-10]-sim.links[1].cumulative_outflows[t] for t in range(10, len(times_p))]
q_1 = [0 for _ in range(9)] + q_1_fol

q_2_fol = [sim.links[2].cumulative_inflows[t-10]-sim.links[2].cumulative_outflows[t] for t in range(10, len(times_p))]
q_2 = [0 for _ in range(9)] + q_2_fol



#axs.step(sim.get_times(0), f0)

axs[0].step(sim.get_times(0), f1, color=color_1, label='Left Flow')
axs[0].step(sim.get_times(0), f2, color=color_2, label='Opposing Through Flow')
#axs[0].step(sim.get_times(0), f0, label='X')
axs[0].step(times_demands, dem_2, color=color_1, alpha=0.4, linestyle='--', label='Left dem.', linewidth=3)
axs[0].step(times_demands, dem_3, color=color_2, alpha=0.4, linestyle='--', label='Opp. Through dem.', linewidth=3)


axs[1].step(sim.get_times(0), q_1, color=color_1, label='Left')
axs[1].step(sim.get_times(0), q_2, color=color_2, label='Opposing Through')
#axs[1].step(sim.get_times(0), q_0, label='X')

axs[0].set_ylabel('Link Outflow (veh)')
axs[1].set_ylabel('Queue (veh)')

ticks_protected = [0, 60, 120, 180]
ticks_protected_end = [8, 68, 128, 188]
ticks_permitted_end = [40, 100, 160, 220]

for i in range(2):
    for t in ticks_protected+ticks_protected_end:
        
        axs[i].axvline(t, color=color_1, linestyle='-', alpha=0.8)
    


    for t in ticks_protected_end+ticks_permitted_end:
        axs[i].axvline(t+0.5, color=color_1, linestyle='--', alpha=0.8)

axs[0].set_ylim((0,2))
axs[0].set_yticks([0,1,2])
axs[1].set_ylim((0,15))
axs[1].set_yticks([0,5,10,15])

for i in range(2):
    axs[i].grid()
    axs[i].set_xlim(0, 240)
    axs[i].set_xlabel('Time (s)')
    axs[i].legend(fontsize=7)
    axs[i].set_xticks([0,60,120,180,240])
    axs[i].set_xlim((0,240))
#     axs[i].set_ylabel('Cumulative Outflow (veh)')

for t in ticks_protected:
    axs[0].annotate('P', xy=(t+2.5, 1.8), xytext=(t+2.5, 1.8), xycoords='data', fontsize=12, rotation=0)

for t in ticks_protected:
    axs[0].annotate('Perm.', xy=(t+15, 1.8), xytext=(t+15, 1.8), xycoords='data', fontsize=12, rotation=0)

axs[0].set_title('(a)')
axs[1].set_title('(b)')

pylab.tight_layout()
pylab.savefig(figures_output_path + "signalized.pdf", dpi=600)
pylab.show()
